In [ ]:
import pandas as pd

In [ ]:
# Load in previously processed dataset
application_train = pd.read_csv('./home-credit-train.csv')
application_train.shape

In [ ]:
application_valid = pd.read_csv('./home-credit-valid.csv')
application_valid.shape

In [ ]:
application = pd.concat([application_train, application_valid], axis=0)

In [ ]:
application.drop(labels=['Unnamed: 0'], inplace=True)

In [ ]:
application.shape

In [ ]:
rows = application.shape[0]

In [ ]:
prev_app = pd.read_csv('./previous_application.csv', nrows=rows)
prev_app = prev_app.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop=True)
prev_app = prev_app[prev_app['SK_ID_CURR'].isin(application['SK_ID_CURR'].values.tolist())]
prev_app.shape

In [ ]:
bureau = pd.read_csv('./bureau.csv', nrows=rows)
bureau = bureau.sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop=True)
bureau.shape

In [ ]:
bureau_balance = pd.read_csv('./bureau_balance.csv', nrows=rows)
bureau_balance = bureau_balance.sort_values('SK_ID_BUREAU').reset_index(drop = True)
bureau_balance.shape

In [ ]:
bureau_merged = pd.merge(bureau, bureau_balance, on=['SK_ID_BUREAU'])
bureau_merged.shape

In [ ]:
bureau_merged = bureau_merged[bureau_merged['SK_ID_CURR'].isin(application['SK_ID_CURR'].values.tolist())]

In [ ]:
credit_balance = pd.read_csv('./credit_card_balance.csv', nrows=rows)
credit_balance = credit_balance.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
credit_balance.shape

In [ ]:
credit_balance = credit_balance[credit_balance['SK_ID_CURR'].isin(application['SK_ID_CURR'].values.tolist())]

In [ ]:
pos_cash = pd.read_csv('./POS_CASH_balance.csv', nrows=rows)
pos_cash = pos_cash.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
pos_cash.shape

In [ ]:
pos_cash = pos_cash[pos_cash['SK_ID_CURR'].isin(application['SK_ID_CURR'].values.tolist())]

In [ ]:
installments = pd.read_csv('./installments_payments.csv', nrows=rows)
installments = installments.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
installments.shape

In [ ]:
installments = installments[installments['SK_ID_CURR'].isin(application['SK_ID_CURR'].values.tolist())]

In [ ]:
dataframes = [prev_app, bureau_merged, installments, pos_cash, credit_balance]

In [ ]:
from functools import reduce

df_merged = reduce(lambda left, right: pd.merge(left,right,on=['SK_ID_CURR'], how='outer'), dataframes)

In [ ]:
df_merged.shape

In [ ]:
df_merged.to_csv('home-credit-df-merged.csv', index=False)

In [ ]:
numerical_features = df_merged.select_dtypes(exclude=['object'])

In [ ]:
categorical_features = df_merged.select_dtypes(exclude=['int', 'float'])
categorical_features.shape

In [ ]:
tmp_categorical = pd.get_dummies(categorical_features)

In [ ]:
pt_1_float_df = application.select_dtypes(include=['float'])
pt_2_float_df = numerical_features.select_dtypes(include=['float'])

pt_1_float_df['SK_ID_CURR'] = application['SK_ID_CURR']
pt_2_float_df['SK_ID_CURR'] = df_merged['SK_ID_CURR']

float_merged = pd.merge(pt_1_float_df, pt_2_float_df, on=['SK_ID_CURR'])
float_merged.shape

In [ ]:
pt_1_int_df = application.select_dtypes(include=['int'])
pt_2_int_df = numerical_features.select_dtypes(include=['int'])

pt_1_int_df['SK_ID_CURR'] = application['SK_ID_CURR']
pt_2_int_df['SK_ID_CURR'] = df_merged['SK_ID_CURR']

int_merged = pd.merge(pt_1_int_df, pt_2_int_df, on=['SK_ID_CURR'])
int_merged.drop(labels=['TARGET'], inplace=True, axis=1)

In [ ]:
int_merged.shape

In [ ]:
float_merged = float_merged.fillna(method='ffill')
float_merged = float_merged.fillna(method='bfill')

In [ ]:
float_nans = float_merged.columns[float_merged.isnull().any().tolist()].tolist()
float_merged.drop(labels=float_nans, axis=1, inplace=True)

In [ ]:
IDS = float_merged['SK_ID_CURR']

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaled = scaler.fit_transform(float_merged.drop('SK_ID_CURR', axis=1))
normalized_float_df = pd.DataFrame(scaled)

In [ ]:
normalized_float_df.shape

In [ ]:
normalized_float_df['SK_ID_CURR'] = IDS
normalized_float_df.columns = float_merged.columns

In [ ]:
normalized_float_df.to_csv('home-credit-float-features.csv', index=False)

In [ ]:
# Add automated features back into the dataset
autofeature = pd.read_csv("../home-credit-float-autofeatures.csv")
autofeature = pd.concat([IDS, autofeature], axis=1)
autofeature.shape

In [ ]:
# Add manual features back into the dataset
manual_features = pd.read_csv("./home-credit-manual-float-features.csv")
manual_features = pd.concat([IDS, manual_features], axis=1)
manual_features.shape

In [ ]:
int_merged.shape

In [ ]:
categorical_features.shape

In [ ]:
# Autofeatures
features_A = pd.concat([
    autofeature,
    tmp_categorical,
    int_merged.drop('SK_ID_CURR', axis=1)], axis=1)

In [ ]:
# Manual Features
features_B = pd.concat([
    manual_features,
    tmp_categorical,
    int_merged.drop('SK_ID_CURR', axis=1)], axis=1)

In [ ]:
app = application[['SK_ID_CURR', 'TARGET']]

In [ ]:
features_A_final = pd.merge(features_A, app, on='SK_ID_CURR', how='left')

In [ ]:
features_B_final = pd.merge(features_B, app, on='SK_ID_CURR', how='left')

In [ ]:
features_final['TARGET'].unique()

In [ ]:
from sklearn.model_selection import train_test_split
 
def split_data(df):
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('TARGET', axis=1), df['TARGET'], test_size=0.33, random_state=42)
    return pd.concat([X_train, y_train], axis=1), pd.concat([X_test, y_test], axis=1)

In [ ]:
def write_to_file(df, outfile):
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.to_csv(outfile, index=False)

In [ ]:
train_A, valid_A = split_data(features_A_final)
train_B, valid_B = split_data(features_B_final)

In [ ]:
write_to_file(train_A, 'home-credit-manual-processed-train.csv')
write_to_file(valid_A, 'home-credit-manual-processed-valid.csv')
write_to_file(train_B, 'home-credit-auto-processed-train.csv')
write_to_file(valid_B, 'home-credit-auto-processed-valid.csv')